In [1]:
from time import sleep
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from robotshyu import Robot as rob
from tasho.OCPGenerator import OCPGenerator
import numpy as np
import casadi as cs
import tasho
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env

In [2]:
robot = rob.Robot("kinova")
link_name = 7
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 1, 0, 0.6], [1, 0, 0, 0.0], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [1.0193752249977548, -0.05311582280659044, -2.815452580946695,
    1.3191046402052224, 2.8582660722530533, 1.3988994390898029, 1.8226311094569714]
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 7-DoF robot: kinova


In [3]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [4]:
task_P2P.write_task_graph("P2P_Task.svg")

In [5]:
reg_jacc = task_P2P.constraint_expressions['reg_qdd_'+robot.name].change_weight(0.1)
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1))

In [6]:
task_P2P.write_task_graph("P2P_Task2.svg")

In [7]:
horizon_steps = 10
horizon_period = 3.0
OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      529
Number of nonzeros in inequality constraint Jacobian.:      217
Number of nonzeros in Lagrangian Hessian.............:      168

Total number of variables............................:      224
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      164
Total number of inequality constrai

In [8]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

from tasho import WorldSimulator
import pybullet as p

obj = WorldSimulator.WorldSimulator()

    # Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 3, 4, 5, 6]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

pybullet build time: May 20 2022 19:44:17


Ending simulation


Tunnel-following task

In [9]:
from tasho.templates.SE3_tunnel import SE3Tunnel

vel_limit = 0.5 #m/s
acc_limit = 2.0 #m/s^2

trans_tunnel_size = 0.02
rot_tunnel_size = 5/3.14159/180

link_name = 7
a_p = 0.2
z_p = 0.1
SE3_path_fun = lambda s : cs.vertcat(
                    cs.horzcat(cs.DM([[0, 1, 0], [1, 0, 0], [0, 0, -1]]), 
                    cs.vertcat(0.6+z_p*cs.sin(s*(4*np.pi)), 0.0+a_p*cs.sin(s*(2*np.pi)), 0.25+a_p*cs.sin(s*(2*np.pi))*cs.cos(s*(2*np.pi)))), 
                    cs.DM([0, 0, 0, 1]).T)
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[-1, 0, 0, 0.6], [0, 1, 0, 0.35], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [ 0.42280387,  1.56128753, -2.07387664,  1.1543891,   1.7809308,   2.03112421, 4.02677039]

tunnel_disembodiedEE = SE3Tunnel("contouring", SE3_path_fun, vel_limit, acc_limit, trans_tunnel_size, rot_tunnel_size)
tunnel_disembodiedEE.write_task_graph("tunnel_disEE.svg")

In [10]:
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

# # Using the template to create the P2P task
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)
# Removing the goal pose of P2P because not relevant for tunnel-following
task_P2P.remove_expression(goal_pose)
task_P2P.write_task_graph("P2P_Task3.svg")

In [11]:
tunnel_disembodiedEE.substitute_expression(tunnel_disembodiedEE.variables['SE3_traj_contouring'], task_P2P.expressions["pose_7_kinova"])
tunnel_disembodiedEE.write_task_graph("tunnel_disEE2.svg")

In [12]:
tunnel_task = TaskModel.compose("lemniscate", "tunnel_following", tunnel_disembodiedEE, task_P2P)
tunnel_task.write_task_graph("tunnel_following.svg")

In [13]:
horizon_steps = 30
horizon_period = 3
OCP_gen = OCPGenerator(tunnel_task, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)

OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}} 
)

OCP_gen.tc.solve_ocp()

st = OCP_gen.stage_tasks[0]
# print(OCP_gen.tc.sol_sample(st.expressions['SE3_path_contouring'].x))
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
# print(qsol)
t_grid, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)


This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     1705
Number of nonzeros in inequality constraint Jacobian.:      720
Number of nonzeros in Lagrangian Hessian.............:     1320

Total number of variables............................:      736
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      505
Total number of inequality constraints...............:      720
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      600
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.5000321e-05 1.00e+00 6.06e-04  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
star

In [14]:
obj = WorldSimulator.WorldSimulator()

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 3, 4, 5, 6]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

Ending simulation


Object picking MPC

In [15]:
# Define initial conditions of the robot
q0_val = [0, -0.523598, 0, 2.51799, 0, -0.523598, -1.5708]
qd0_val = [0] * robot.ndof

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, 7, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("P2P_Task4.svg")

In [16]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_kinova'])
task_P2P.remove_terminal_constraints('rot_con_pose_7_kinova_vs_goal',
                                    'trans_con_pose_7_kinova_vs_goal')
task_P2P.write_task_graph("P2P_Task5.svg")

In [17]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_kinova']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_7_kinova_vs_goal'], 1.0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_7_kinova_vs_goal'], 1), 
                            Regularization(task_P2P.variables['qd_kinova'], 1e-4))

task_P2P.write_task_graph("P2P_Task6.svg")

In [18]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_kinova'].x
qd = pOCP.stage_tasks[0].variables['qd_kinova'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_kinova'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_kinova'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_kinova'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_kinova", "var":"qd_kinova", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_7_kinova_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
import os; os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,846]:

b3Printf: /home/mtplnr/mpc_ws/postech_lec/.venv/lib/python3.8/site-packages/robotshy

sh: 1: export: : bad variable name


u/robots/Kinova/Gen3/kortex_description/urdf/: cannot extract mesh from '/home/mtplnr/mpc_ws/postech_lec/.venv/lib/python3.8/site-packages/robotshyu/robots/Kinova/Gen3/kortex_description/meshes/end_effector_link.STL'

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,1166]:

b3Printf: /home/mtplnr/mpc_ws/postech_lec/.venv/lib/python3.8/site-packages/robotshyu/robots/Kinova/Gen3/kortex_description/urdf/: cannot extract anything useful from mesh '/home/mtplnr/mpc_ws/postech_lec/.venv/lib/python3.8/site-packages/robotshyu/robots/Kinova/Gen3/kortex_description/meshes/end_effector_link.STL'

b3Printf: b3Warning[examples/SharedMemory/plugins/tinyRendererPlugin/TinyRendererVisualShapeConverter.cpp,558]:

b3Printf: issue extracting mesh from COLLADA/STL file /home/mtplnr/mpc_ws/postech_lec/.venv/lib/python3.8/site-packages/robotshyu/robots/Kinova/Gen3/kortex_description/meshes/end_effector_link.STL

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
T

In [19]:
vars_db = tc.generate_MPC_component("./", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti2_p_2[12:15]-fk_T(opti2_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti2_p_2[12:15]-fk_T(opti2_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [20]:
MPC_component = MPC("kinova_obj_pickup", "./" + tc.name + ".json")

In [21]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
kinovaID = obj.add_robot(position, orientation, "kinova")

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5, 6]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetJointState(kinovaID, joint_indices, q0_val)
obj.setController(
    kinovaID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

In [ ]:
# Execute the MPC loop
q_log = []
q_dot_log = []
predicted_pos_log = []

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(kinovaID, joint_indices)
    qd_now = obj.readJointVelocities(kinovaID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_kinova"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_kinova"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 1, 0, predicted_pos[0]]),
        cs.hcat([1, 0, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_kinova"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_kinova"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_kinova"]["val"] * t_mpc).full()
    obj.setController(
        kinovaID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.

----------- MPC execution -----------
Predicted position of cube [0.49999968066805, 0.31677326454043, 0.27249110475156]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 157.00us ( 39.25us) 156.67us ( 39.17us)         4
       nlp_g  |  33.00us (  8.25us)  33.81us (  8.45us)         4
    nlp_grad  | 134.00us (134.00us) 134.30us (134.30us)         1
  nlp_grad_f  | 425.00us (106.25us) 427.03us (106.76us)         4
  nlp_hess_l  |   1.89ms (630.67us)   1.89ms (631.58us)         3
   nlp_jac_g  |  52.00us ( 13.00us)  51.92us ( 12.98us)         4
       total  |  12.32ms ( 12.32ms)   4.31ms (  4.31ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999968506725, 0.32652397891008, 0.27249052544209]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 965.00us (241.25us) 179.48us ( 44.87us)         4
       nlp_g  |   1.22ms (305.00us)  57.84us ( 14.46us)         4
    nlp_grad  | 160.00us (160.00us

----------- MPC execution -----------
Predicted position of cube [0.49999522971114, 0.43300391338629, 0.084989917541916]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 155.00us ( 38.75us) 157.10us ( 39.28us)         4
       nlp_g  |  46.00us ( 11.50us)  46.13us ( 11.53us)         4
    nlp_grad  | 133.00us (133.00us) 133.42us (133.42us)         1
  nlp_grad_f  | 452.00us (113.00us) 453.08us (113.27us)         4
  nlp_hess_l  |   9.88ms (  3.29ms)   1.90ms (631.80us)         3
   nlp_jac_g  |  51.00us ( 12.75us)  51.82us ( 12.96us)         4
       total  |  12.60ms ( 12.60ms)   4.60ms (  4.60ms)         1
----------- MPC execution -----------
Predicted position of cube [0.4999950969829, 0.43465349718153, 0.084989958498636]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 239.00us ( 47.80us) 238.80us ( 47.76us)         5
       nlp_g  |  49.00us (  9.80us)  48.07us (  9.61us)         5
    nlp_grad  | 132.00us (132.00u

----------- MPC execution -----------
Predicted position of cube [0.49999354750716, 0.43634547411444, 0.084989987246302]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   2.58ms (859.00us) 151.93us ( 50.64us)         3
       nlp_g  |  33.00us ( 11.00us)  32.18us ( 10.73us)         3
    nlp_grad  | 190.00us (190.00us) 192.20us (192.20us)         1
  nlp_grad_f  | 406.00us (135.33us) 399.68us (133.23us)         3
  nlp_hess_l  |   1.59ms (795.00us)   1.59ms (793.21us)         2
   nlp_jac_g  |  91.00us ( 30.33us)  51.87us ( 17.29us)         3
       total  |  10.80ms ( 10.80ms)   3.94ms (  3.94ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354691867, 0.43634546654046, 0.084989995310537]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 143.00us ( 47.67us) 144.35us ( 48.12us)         3
       nlp_g  |  43.00us ( 14.33us)  43.47us ( 14.49us)         3
    nlp_grad  | 171.00us (171.00

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214853, 0.084989999999964]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 117.00us ( 39.00us) 116.33us ( 38.78us)         3
       nlp_g  |  27.00us (  9.00us)  27.36us (  9.12us)         3
    nlp_grad  | 133.00us (133.00us) 133.82us (133.82us)         1
  nlp_grad_f  | 317.00us (105.67us) 316.44us (105.48us)         3
  nlp_hess_l  |   1.27ms (635.00us)   1.27ms (635.70us)         2
   nlp_jac_g  |  38.00us ( 12.67us)  38.20us ( 12.73us)         3
       total  |   3.21ms (  3.21ms)   3.20ms (  3.20ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214854, 0.084989999999972]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 158.00us ( 52.67us) 158.78us ( 52.93us)         3
       nlp_g  |  58.00us ( 19.33us)  58.73us ( 19.58us)         3
    nlp_grad  | 163.00us (163.00

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 141.00us ( 47.00us) 141.66us ( 47.22us)         3
       nlp_g  |  38.00us ( 12.67us)  38.19us ( 12.73us)         3
    nlp_grad  | 161.00us (161.00us) 160.72us (160.72us)         1
  nlp_grad_f  | 384.00us (128.00us) 384.22us (128.07us)         3
  nlp_hess_l  |   1.59ms (795.00us)   1.59ms (795.67us)         2
   nlp_jac_g  |  47.00us ( 15.67us)  47.74us ( 15.91us)         3
       total  |  10.75ms ( 10.75ms)   3.86ms (  3.86ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 408.00us (136.00us) 144.82us ( 48.27us)         3
       nlp_g  |  46.00us ( 15.33us)  46.19us ( 15.40us)         3
    nlp_grad  | 649.00us (649.00us) 187.68us (187.68

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   2.28ms (761.00us) 144.32us ( 48.11us)         3
       nlp_g  |  50.00us ( 16.67us)  49.75us ( 16.58us)         3
    nlp_grad  |   6.37ms (  6.37ms) 174.27us (174.27us)         1
  nlp_grad_f  |   2.14ms (712.00us) 439.84us (146.61us)         3
  nlp_hess_l  |  16.46ms (  8.23ms)   1.60ms (801.25us)         2
   nlp_jac_g  |  49.00us ( 16.33us)  49.25us ( 16.42us)         3
       total  |  42.96ms ( 42.96ms)   4.17ms (  4.17ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 136.00us ( 45.33us) 136.86us ( 45.62us)         3
       nlp_g  |  37.00us ( 12.33us)  37.48us ( 12.49us)         3
    nlp_grad  | 135.00us (135.00us) 134.55us (134.55

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 144.00us ( 48.00us) 145.02us ( 48.34us)         3
       nlp_g  |  47.00us ( 15.67us)  47.84us ( 15.95us)         3
    nlp_grad  | 164.00us (164.00us) 164.09us (164.09us)         1
  nlp_grad_f  | 401.00us (133.67us) 402.07us (134.02us)         3
  nlp_hess_l  |   1.59ms (796.00us)   1.59ms (796.54us)         2
   nlp_jac_g  |  49.00us ( 16.33us)  49.02us ( 16.34us)         3
       total  |  11.77ms ( 11.77ms)   4.08ms (  4.08ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 578.00us (192.67us) 144.29us ( 48.10us)         3
       nlp_g  |  63.00us ( 21.00us)  39.72us ( 13.24us)         3
    nlp_grad  | 164.00us (164.00us) 163.66us (163.66

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 971.00us (323.67us) 179.42us ( 59.81us)         3
       nlp_g  |  42.00us ( 14.00us)  42.23us ( 14.08us)         3
    nlp_grad  | 202.00us (202.00us) 202.69us (202.69us)         1
  nlp_grad_f  | 496.00us (165.33us) 497.61us (165.87us)         3
  nlp_hess_l  |  10.46ms (  5.23ms)   1.93ms (962.92us)         2
   nlp_jac_g  |   3.86ms (  1.29ms)  60.05us ( 20.02us)         3
       total  |  26.58ms ( 26.58ms)   5.10ms (  5.10ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 328.00us (109.33us) 168.89us ( 56.30us)         3
       nlp_g  |  71.00us ( 23.67us)  48.41us ( 16.14us)         3
    nlp_grad  | 164.00us (164.00us) 164.67us (164.67

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 234.00us ( 78.00us) 156.36us ( 52.12us)         3
       nlp_g  |  41.00us ( 13.67us)  41.02us ( 13.67us)         3
    nlp_grad  | 166.00us (166.00us) 161.66us (161.66us)         1
  nlp_grad_f  |   1.81ms (602.33us) 418.16us (139.39us)         3
  nlp_hess_l  |  24.53ms ( 12.26ms)   1.65ms (823.22us)         2
   nlp_jac_g  | 531.00us (177.00us)  49.97us ( 16.66us)         3
       total  |  49.05ms ( 49.05ms)   4.30ms (  4.30ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 845.00us (281.67us) 160.24us ( 53.41us)         3
       nlp_g  | 674.00us (224.67us)  37.57us ( 12.52us)         3
    nlp_grad  | 318.00us (318.00us) 159.61us (159.61

In [ ]:
obj.end_simulation()

max(MPC_component.solver_times)